In [1]:
import ee
import geemap
import datetime
import numpy as np
import pandas as pd

In [2]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')

landsat_start_year = 1990 # с 1990 тепловая информация Landsat-5 без колебаний
modis_date_start = ee.Date("2001-01-01") # в 2001 начал предоставлять тепловую информацию Terra
landsat_date_start = ee.Date(f"{landsat_start_year}-03-16")
modis_start_year = modis_date_start.get("year").getInfo()
date_end = datetime.datetime.now()
end_year = date_end.date().year
openwater_month_start = 6 # июнь
openwater_month_end = 10 # октябрь

baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

(node:25893) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!


# LANDSAT

In [6]:
LandsatLST = geemap.requireJS("../baikal_vkr_init/Landsat_LST.js")

In [4]:
use_ndvi = False

def get_landsat_collection(satellite_name: str):
   coll = LandsatLST.collection(satellite_name, landsat_date_start, date_end, baikal_shape, use_ndvi)
   return coll.select(['LST'])

In [7]:
landsats_collection = ee.ImageCollection.merge(get_landsat_collection('L8'),get_landsat_collection('L9'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L7'))
landsats_collection = ee.ImageCollection.merge(landsats_collection,get_landsat_collection('L5'))

In [ ]:
def get_average_lst_landsat(landsat_collection: ee.ImageCollection, start_time, end_time):
   temp = landsat_collection.filterDate(start_time, end_time).mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return result_dict['LST']

In [7]:
lst_landsat = {}

for year in range(landsat_start_year, end_year):
   lst_landsat[year] = {}
   for month in range(openwater_month_start, openwater_month_end):
      start_date = ee.Date(f"{year}-{month}-1")
      end_date = ee.Date(f"{year}-{(month+1)}-1")
      
      result = get_average_lst_landsat(landsats_collection, start_date, end_date)
      if(result is None):
         continue
      lst_landsat[year][month] = result
   
   lst_landsat[year] = np.mean(list(lst_landsat[year].values()))
   if lst_landsat[year] is None:
      del lst_landsat[year]

lst_landsat

{1990: 9.94151105813071,
 1991: 8.670243772634592,
 1992: -2.0105041979387375,
 1993: 7.956542834155669,
 1994: 9.34452038501193,
 1995: 8.04470759348891,
 1996: 7.082734448761985,
 1997: 8.935313905816676,
 1998: 9.281472002444602,
 1999: 7.613358335015398,
 2000: 9.067563007508321,
 2001: 6.577084616749362,
 2002: 12.944188352610443,
 2003: 7.712404849028003,
 2004: 7.702411146961344,
 2005: 9.258553590605247,
 2006: 7.586450930492356,
 2007: 10.296707394361517,
 2008: 8.111687043729976,
 2009: 8.053128114881378,
 2010: 7.317162556263412,
 2011: 8.040792549218494,
 2012: 10.04538712944025,
 2013: 10.489422570326163,
 2014: 12.669070935022372,
 2015: 12.110493647292312,
 2016: 10.845184720867204,
 2017: 10.18566067643681,
 2018: 10.015904896912351,
 2019: 10.529639238809038,
 2020: 12.717650167983859,
 2021: 8.898396186200852,
 2022: 8.728638766197365,
 2023: 9.901500828971503,
 2024: 10.490878109042882}

# L2SP

In [35]:
def Cloudmask(image: ee.Image):
   qa = image.select('QA_PIXEL')
   mask = qa.bitwiseAnd(1 << 3).Or(qa.bitwiseAnd(1 << 4))
   return image.updateMask(mask.Not())

def L57_scale_to_celsius(image: ee.image):
   scaled_lst = image.select(['ST_B6']).multiply(0.00341802).subtract(124.15).rename(['LST'])
   return image.addBands(srcImg=scaled_lst)

def L89_scale_to_celsius(image: ee.image):
   scaled_lst = image.select(['ST_B10']).multiply(0.00341802).subtract(124.15).rename(['LST'])
   return image.addBands(srcImg=scaled_lst)

def get_l2sp_list(satellite_name: str):
   match satellite_name:
      case 'Landsat 5 Tier 1':
         coll = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").map(lambda img: L57_scale_to_celsius(img))
      case 'Landsat 7 Tier 1':
         coll = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").map(lambda img: L57_scale_to_celsius(img))
      case 'Landsat 8 Tier 1':
         coll = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(lambda img: L89_scale_to_celsius(img))
      case 'Landsat 9 Tier 1':
         coll = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').map(lambda img: L89_scale_to_celsius(img))
      case 'Landsat 5 Tier 2':
         coll = ee.ImageCollection("LANDSAT/LT05/C02/T2_L2").map(lambda img: L57_scale_to_celsius(img))
      case 'Landsat 7 Tier 2':
         coll = ee.ImageCollection("LANDSAT/LE07/C02/T2_L2").map(lambda img: L57_scale_to_celsius(img))
      case 'Landsat 8 Tier 2':
         coll = ee.ImageCollection('LANDSAT/LC08/C02/T2_L2').map(lambda img: L89_scale_to_celsius(img))
      case 'Landsat 9 Tier 2':
         coll = ee.ImageCollection('LANDSAT/LC09/C02/T2_L2').map(lambda img: L89_scale_to_celsius(img))

   coll = coll.filter(
            ee.Filter.date(landsat_date_start, date_end)
         ).filterBounds(
            baikal_shape
         ).filter(
            "PROCESSING_LEVEL == 'L2SP'"
         ).map(Cloudmask)
   return coll.select(['LST'])

In [36]:
l2sp_collection = ee.ImageCollection.merge(get_l2sp_list('Landsat 5 Tier 1'), get_l2sp_list('Landsat 7 Tier 1'))
l2sp_collection = ee.ImageCollection.merge(l2sp_collection,get_l2sp_list('Landsat 8 Tier 1'))
l2sp_collection = ee.ImageCollection.merge(l2sp_collection,get_l2sp_list('Landsat 9 Tier 1'))
l2sp_collection = ee.ImageCollection.merge(l2sp_collection,get_l2sp_list('Landsat 5 Tier 2'))
l2sp_collection = ee.ImageCollection.merge(l2sp_collection,get_l2sp_list('Landsat 7 Tier 2'))
l2sp_collection = ee.ImageCollection.merge(l2sp_collection,get_l2sp_list('Landsat 8 Tier 2'))
l2sp_collection = ee.ImageCollection.merge(l2sp_collection,get_l2sp_list('Landsat 9 Tier 2'))

In [30]:
def get_average_lst_l2sp(landsat_collection: ee.ImageCollection, start_time, end_time):
   temp = landsat_collection.filterDate(start_time, end_time).mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return result_dict['LST']

In [37]:
lst_l2sp = {}

for year in range(landsat_start_year, end_year):
   lst_l2sp[year] = {}
   for month in range(openwater_month_start, openwater_month_end):
      start_date = ee.Date(f"{year}-{month}-1")
      end_date = ee.Date(f"{year}-{(month+1)}-1")
      
      result = get_average_lst_l2sp(l2sp_collection, start_date, end_date)
      if(result is None):
         continue
      lst_l2sp[year][month] = result
   
   lst_l2sp[year] = np.mean(list(lst_l2sp[year].values()))
   if lst_l2sp[year] is None:
      del lst_l2sp[year]

lst_l2sp

{1990: 11.52196755603775,
 1991: 12.404069463328623,
 1992: 1.6301826964997548,
 1993: 13.145985911690591,
 1994: 11.971119403324407,
 1995: 10.789324301379168,
 1996: 12.42958903567926,
 1997: 8.995261763169525,
 1998: 12.081733235457616,
 1999: 11.962635338329502,
 2000: 11.939753007159085,
 2001: 11.166436577670936,
 2002: 15.15550877409544,
 2003: 13.099789776063934,
 2004: 11.685261355746018,
 2005: 12.529072145691176,
 2006: 10.524219489258215,
 2007: 13.672654217178705,
 2008: 12.181358547635265,
 2009: 12.120598095672781,
 2010: 11.21029555890743,
 2011: 11.709370510099847,
 2012: 12.379641500319076,
 2013: 13.983231084379739,
 2014: 15.852608257964782,
 2015: 15.412599409461055,
 2016: 15.150619009534738,
 2017: 13.94550969702493,
 2018: 13.409365645594294,
 2019: 14.425735607219298,
 2020: 15.001575481166673,
 2021: 13.19294392786191,
 2022: 12.117687221027202,
 2023: 12.34562721541754,
 2024: 9.63801515295418}

## То же, но по предподсчитанным значениям. 
НЕКОРРЕКТНО

In [8]:
def add_timezone(df: pd.DataFrame) -> pd.DataFrame:
   format = "%Y-%m-%d %H:%M:%S.%f"
   df["Date"] = df["Date"].apply(lambda dt: pd.to_datetime(dt, format=format))

In [10]:
all_years_landsat_df = pd.read_csv('../Валидация/Данные_валидации/landsat_baikal_lst.csv')
add_timezone(all_years_landsat_df)
all_years_landsat_df

,Date,Image T
0,1986-04-29 03:23:24.899000000,-1.539700
1,1986-05-31 03:22:32.065063000,2.058033
2,1986-06-02 03:08:55.313094000,-3.599559
3,1986-06-02 03:09:19.171038000,-0.556575
4,1986-06-09 03:16:04.867050000,-0.163581
...,...,...
10012,2025-04-19 03:38:04.276009000,-8.837142
10013,2025-04-19 03:38:28.158577000,-8.964933
10014,2025-04-19 03:38:52.032674000,-9.826516
10015,2025-04-19 03:39:15.906770000,-2.215253


In [11]:
def get_average_lst_landsat_local(landsat_df, start_date, end_date):
   in_bounds_entries = landsat_df[(start_date <= landsat_df["Date"]) & (landsat_df["Date"] <= end_date)]
   return in_bounds_entries["Image T"].mean()

In [17]:
lst_landsat_local = {}
for year in range(landsat_start_year, end_year):
      start_date = datetime.datetime(year=year, month=openwater_month_start, day=1)
      end_date = datetime.datetime(year=year, month=openwater_month_end, day=31)
      
      result = get_average_lst_landsat_local(all_years_landsat_df, start_date, end_date)
      lst_landsat_local[year] = result

lst_landsat_local

{1986: 8.759663685460412,
 1987: 2.7700559934364506,
 1988: 3.40586823059557,
 1989: 7.997293070931044,
 1990: 7.856037313254132,
 1991: 7.3680161728764,
 1992: 3.3140336063937172,
 1993: 7.15149218132415,
 1994: 7.119435288776228,
 1995: 5.607646267554436,
 1996: 6.120807928038543,
 1997: 5.365743478094312,
 1998: 7.2645621714384125,
 1999: 6.442740172669513,
 2000: 7.456334950729774,
 2001: 7.056328889681249,
 2002: 9.955916605597425,
 2003: 7.975299702742071,
 2004: 6.637760675388121,
 2005: 7.597102759071141,
 2006: 6.014261846680311,
 2007: 9.493502600364673,
 2008: 7.0564371466133675,
 2009: 7.513282902079464,
 2010: 6.969347278839847,
 2011: 7.196237246273308,
 2012: 9.224797200496177,
 2013: 7.9906726291247745,
 2014: 9.232270771124375,
 2015: 9.417395202941613,
 2016: 8.558806286942847,
 2017: 7.591164193617533,
 2018: 7.988809488128339,
 2019: 9.066400822576428,
 2020: 9.120943260191137,
 2021: 7.928572549612589,
 2022: 7.384870612306152,
 2023: 8.745032440903028,
 2024: 10.0

In [ ]:
lst_landsat_s = { 1990: 9.879765511679397,
 1991: 8.954173718890887,
 1992: 4.1181207001714135,
 1993: 10.058721180744957,
 1994: 9.22601766201506,
 1995: 7.868444791330705,
 1996: 7.615046916830711,
 1997: 6.4652446881016505,
 1998: 8.29735257295523,
 1999: 7.466276454270664,
 2000: 8.444648433179971,
 2001: 6.8573912127145835,
 2002: 11.29452174088983,
 2003: 8.133507938438473,
 2004: 7.756460249341295,
 2005: 8.815491983317168,
 2006: 6.796633766422254,
 2007: 9.983745663150753,
 2008: 8.444456981800055,
 2009: 7.979558202903978,
 2010: 7.409646087737466,
 2011: 7.802575171575934,
 2012: 8.998809616805136,
 2013: 9.029895691880107,
 2014: 10.268777774431134,
 2015: 10.722652392625395,
 2016: 10.075703142279956,
 2017: 9.112241223415538,
 2018: 9.1208041147658,
 2019: 9.838905106835822,
 2020: 10.8137280883284,
 2021: 8.059809265577483,
 2022: 8.255228706723798,
 2023: 9.533078486705547,
 2024: 9.525957321098034}

l2sp_s = {1990: 11.52196755603775,
 1991: 12.404069463328623,
 1992: 8.643606859163437,
 1993: 13.145985911690591,
 1994: 14.048962140952437,
 1995: 11.345508528558398,
 1996: 12.42958903567926,
 1997: 9.152201421399248,
 1998: 12.081733235457616,
 1999: 11.956025640427717,
 2000: 11.939753007159085,
 2001: 11.193228361224506,
 2002: 15.15550877409544,
 2003: 13.090796408017157,
 2004: 11.685261355746018,
 2005: 12.529072145691176,
 2006: 10.091901503963571,
 2007: 13.684239171630615,
 2008: 12.181358547635265,
 2009: 12.122458028117567,
 2010: 12.00415215393862,
 2011: 11.709370510099847,
 2012: 12.36608233705122,
 2013: 13.983231084379739,
 2014: 15.859958972899399,
 2015: 15.412309450445182,
 2016: 15.89598328372118,
 2017: 13.94550969702493,
 2018: 13.484296553193952,
 2019: 14.425735607219298,
 2020: 15.001575481166673,
 2021: 13.234289244416829,
 2022: 12.146844465354125,
 2023: 12.782916476456382,
 2024: 9.947395378278914}

l2sp_with_tier2_s = {1990: 11.52196755603775,
 1991: 12.404069463328623,
 1992: 1.6301826964997548,
 1993: 13.145985911690591,
 1994: 11.971119403324407,
 1995: 10.789324301379168,
 1996: 12.42958903567926,
 1997: 8.995261763169525,
 1998: 12.081733235457616,
 1999: 11.962635338329502,
 2000: 11.939753007159085,
 2001: 11.166436577670936,
 2002: 15.15550877409544,
 2003: 13.099789776063934,
 2004: 11.685261355746018,
 2005: 12.529072145691176,
 2006: 10.524219489258215,
 2007: 13.672654217178705,
 2008: 12.181358547635265,
 2009: 12.120598095672781,
 2010: 11.21029555890743,
 2011: 11.709370510099847,
 2012: 12.379641500319076,
 2013: 13.983231084379739,
 2014: 15.852608257964782,
 2015: 15.412599409461055,
 2016: 15.150619009534738,
 2017: 13.94550969702493,
 2018: 13.409365645594294,
 2019: 14.425735607219298,
 2020: 15.001575481166673,
 2021: 13.19294392786191,
 2022: 12.117687221027202,
 2023: 12.34562721541754,
 2024: 9.63801515295418}

# MODIS

In [ ]:
def M11_day_Cloudmask(image: ee.Image):
   qa = image.select('QC_Day')
   mask = qa.bitwiseAnd(1 << 4).And(qa.bitwiseAnd(1 << 5))
   return image.updateMask(mask.Not())

def M11_night_Cloudmask(image: ee.Image):
   qa = image.select('QC_Night')
   mask = qa.bitwiseAnd(1 << 4).And(qa.bitwiseAnd(1 << 5))
   return image.updateMask(mask.Not())

def M21_Cloudmask(image: ee.Image):
   qa = image.select('QC')
   mask = qa.bitwiseAnd(1 << 4).And(qa.bitwiseAnd(1 << 5))
   return image.updateMask(mask.Not())

def M11_day_scale_to_celsius(modis_image: ee.image):
   scaled_lsts = modis_image.select(['LST_Day_1km']).multiply(0.02).subtract(273.15).rename(['LST'])
   renamed_time = modis_image.select(['Day_view_time']).rename(['View_Time'])
   result_image = modis_image.addBands(srcImg=scaled_lsts)
   return result_image.addBands(srcImg=renamed_time)

def M11_night_scale_to_celsius(modis_image: ee.image):
   scaled_lsts = modis_image.select(['LST_Night_1km']).multiply(0.02).subtract(273.15).rename(['LST'])
   renamed_time = modis_image.select(['Night_view_time']).rename(['View_Time'])
   result_image = modis_image.addBands(srcImg=scaled_lsts)
   return result_image.addBands(srcImg=renamed_time)

def M21_day_scale_to_celsius(modis_image: ee.image):
   scaled_lsts = modis_image.select(['LST_1KM']).subtract(273.15).rename(['LST'])
   result_image = modis_image.addBands(srcImg=scaled_lsts)
   return result_image

def M21_night_scale_to_celsius(modis_image: ee.image):
   scaled_lsts = modis_image.select(['LST_1KM']).subtract(273.15).rename(['LST'])
   result_image = modis_image.addBands(srcImg=scaled_lsts)
   return result_image

def M_get_coll(satellite_name: str):
   match satellite_name:
      case 'MOD11_Terra_day':
         coll = ee.ImageCollection('MODIS/061/MOD11A2').map(lambda img: M11_day_scale_to_celsius(img))
         coll = coll.map(M11_day_Cloudmask)
      case 'MOD11_Terra_night':
         coll = ee.ImageCollection('MODIS/061/MOD11A2').map(lambda img: M11_night_scale_to_celsius(img))
         coll = coll.map(M11_night_Cloudmask)
      case 'MOD21_Terra_day':
         coll = ee.ImageCollection('MODIS/061/MOD21A1D').map(lambda img: M21_day_scale_to_celsius(img))
         coll = coll.map(M21_Cloudmask)
      case 'MOD21_Terra_night':
         coll = ee.ImageCollection('MODIS/061/MOD21A1N').map(lambda img: M21_night_scale_to_celsius(img))
         coll = coll.map(M21_Cloudmask)
      case 'MOD11_Aqua_day':
         coll = ee.ImageCollection('MODIS/061/MYD11A2').map(lambda img: M11_day_scale_to_celsius(img))
         coll = coll.map(M11_day_Cloudmask)
      case 'MOD11_Aqua_night':
         coll = ee.ImageCollection('MODIS/061/MYD11A2').map(lambda img: M11_night_scale_to_celsius(img))
         coll = coll.map(M11_night_Cloudmask)
      case 'MOD21_Aqua_day':
         coll = ee.ImageCollection('MODIS/061/MYD21A1D').map(lambda img: M21_day_scale_to_celsius(img))
         coll = coll.map(M21_Cloudmask)
      case 'MOD21_Aqua_night':
         coll = ee.ImageCollection('MODIS/061/MYD21A1N').map(lambda img: M21_night_scale_to_celsius(img))
         coll = coll.map(M21_Cloudmask)

   coll = coll.filter(
         ee.Filter.date(modis_date_start, date_end)
      ).filterBounds(
         baikal_shape
      )
   return coll.select(['LST'])

In [58]:
m11_coll = ee.ImageCollection.merge(M_get_coll('MOD11_Terra_day'), M_get_coll('MOD11_Terra_night'))
m11_coll = ee.ImageCollection.merge(m11_coll, M_get_coll('MOD11_Aqua_day'))
m11_coll = ee.ImageCollection.merge(m11_coll, M_get_coll('MOD11_Aqua_night'))
m21_coll = ee.ImageCollection.merge(M_get_coll('MOD21_Terra_day'), M_get_coll('MOD21_Terra_night'))
m21_coll = ee.ImageCollection.merge(m21_coll, M_get_coll('MOD21_Aqua_day'))
m21_coll = ee.ImageCollection.merge(m21_coll, M_get_coll('MOD21_Aqua_night'))

In [43]:
def get_average_lst_modis(modis_coll: ee.ImageCollection, start_time, end_time):
   if(modis_coll.size() == 0):
      return None
   temp = modis_coll.filterDate(start_time, end_time).mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return result_dict['LST']

In [59]:
lst_m21 = {}
lst_m11 = {}

for year in range(modis_start_year, end_year):
   start_date = ee.Date(f"{year}-{openwater_month_start}-1")
   end_date = ee.Date(f"{year}-{openwater_month_end}-1")
   
   result_m11 = get_average_lst_modis(m11_coll, start_date, end_date)
   # print(f"Complete M11 in {start_date} -- {end_date}")
   result_m21 = get_average_lst_modis(m21_coll, start_date, end_date)
   # print(f"Complete M21 in {start_date} -- {end_date}")
   if(result_m11 is not None):
      lst_m11[year] = result_m11
   if(result_m21 is not None):
      lst_m21[year] = result_m21

print(lst_m11, lst_m21)

{2001: 12.178416934189435, 2002: 13.90397137506691, 2003: 11.596507783239726, 2004: 12.224775573501901, 2005: 12.369534468632986, 2006: 11.782929833801525, 2007: 13.117085966760326, 2008: 11.793978815543099, 2009: 11.497500000000029, 2010: 12.021998771067446, 2011: 12.238720739700401, 2012: 12.10122015449441, 2013: 11.33126463014984, 2014: 12.9577914325843, 2015: 14.190030723314635, 2016: 13.868647881554336, 2017: 12.552377399344596, 2018: 12.946748010299656, 2019: 13.368991104868943, 2020: 13.074990344101149, 2021: 12.253575316011261, 2022: 11.059660580524374, 2023: 13.03224777621726, 2024: 13.366857255044128} {2001: 10.739610616019439, 2002: 13.213481788364847, 2003: 10.509448586176326, 2004: 10.460244133403188, 2005: 11.326422122969603, 2006: 9.854122062808523, 2007: 12.047756248339006, 2008: 10.575773063279428, 2009: 10.366565878065071, 2010: 10.518374538414921, 2011: 10.777425399363267, 2012: 10.852779586683317, 2013: 10.288876523972055, 2014: 11.99397934228406, 2015: 13.324569095

In [60]:
lst_m11

{2001: 12.178416934189435,
 2002: 13.90397137506691,
 2003: 11.596507783239726,
 2004: 12.224775573501901,
 2005: 12.369534468632986,
 2006: 11.782929833801525,
 2007: 13.117085966760326,
 2008: 11.793978815543099,
 2009: 11.497500000000029,
 2010: 12.021998771067446,
 2011: 12.238720739700401,
 2012: 12.10122015449441,
 2013: 11.33126463014984,
 2014: 12.9577914325843,
 2015: 14.190030723314635,
 2016: 13.868647881554336,
 2017: 12.552377399344596,
 2018: 12.946748010299656,
 2019: 13.368991104868943,
 2020: 13.074990344101149,
 2021: 12.253575316011261,
 2022: 11.059660580524374,
 2023: 13.03224777621726,
 2024: 13.366857255044128}

In [61]:
lst_m21

{2001: 10.739610616019439,
 2002: 13.213481788364847,
 2003: 10.509448586176326,
 2004: 10.460244133403188,
 2005: 11.326422122969603,
 2006: 9.854122062808523,
 2007: 12.047756248339006,
 2008: 10.575773063279428,
 2009: 10.366565878065071,
 2010: 10.518374538414921,
 2011: 10.777425399363267,
 2012: 10.852779586683317,
 2013: 10.288876523972055,
 2014: 11.99397934228406,
 2015: 13.324569095540856,
 2016: 13.053336477607926,
 2017: 11.987002285251917,
 2018: 11.08621953828368,
 2019: 12.532831005284974,
 2020: 12.023079755911908,
 2021: 10.629778355236144,
 2022: 10.148767181212008,
 2023: 12.225026793951276,
 2024: 12.770074544269605}

In [ ]:
lst_m11_s = {2001: 8.518391334434845,
 2002: 10.562313431334651,
 2003: 7.857616372707389,
 2004: 7.395629688897221,
 2005: 9.14143065926696,
 2006: 7.007350175325124,
 2007: 9.94451399113264,
 2008: 8.258408651177756,
 2009: 8.258186946255545,
 2010: 7.098301657039377,
 2011: 8.083154459303568,
 2012: 9.068294740221727,
 2013: 8.22769966377824,
 2014: 10.313111054749466,
 2015: 10.214666106018871,
 2016: 10.278019374073171,
 2017: 9.111631967910679,
 2018: 9.083092106624825,
 2019: 9.750912104506595,
 2020: 10.673629396456128,
 2021: 8.178589646995642,
 2022: 8.422860681650373,
 2023: 10.092309299156067,
 2024: 9.718802938390988}

lst_m21_s = {2001: 6.523447096502005,
 2002: 11.087026927344537,
 2003: 8.227190985636412,
 2004: 7.918499190633716,
 2005: 9.834607190557838,
 2006: 7.375761008911375,
 2007: 10.603713281690437,
 2008: 8.734014051703918,
 2009: 8.932528225671893,
 2010: 7.71452877253441,
 2011: 8.775993993759588,
 2012: 9.776709236956195,
 2013: 8.83295322142326,
 2014: 10.90875064506874,
 2015: 11.124346848601931,
 2016: 11.072499313254557,
 2017: 9.697003172751204,
 2018: 9.721762029880114,
 2019: 10.577461115660952,
 2020: 11.42118506104802,
 2021: 8.943680470222272,
 2022: 9.072915370205486,
 2023: 11.284890175973027,
 2024: 10.757315581774394}

lst_m11_no_mask = {2001: 12.178416934189435,
 2002: 13.90397137506691,
 2003: 11.596507783239726,
 2004: 12.224775573501901,
 2005: 12.369534468632986,
 2006: 11.782929833801525,
 2007: 13.117085966760326,
 2008: 11.793978815543099,
 2009: 11.497500000000029,
 2010: 12.021998771067446,
 2011: 12.238720739700401,
 2012: 12.10122015449441,
 2013: 11.33126463014984,
 2014: 12.9577914325843,
 2015: 14.190030723314635,
 2016: 13.868647881554336,
 2017: 12.552377399344596,
 2018: 12.946748010299656,
 2019: 13.368991104868943,
 2020: 13.074990344101149,
 2021: 12.253575316011261,
 2022: 11.059660580524374,
 2023: 13.03224777621726,
 2024: 13.366857255044128}

lst_m21_no_mask = {2001: 10.739610616019439,
 2002: 13.213481788364847,
 2003: 10.509448586176326,
 2004: 10.460244133403188,
 2005: 11.326422122969603,
 2006: 9.854122062808523,
 2007: 12.047756248339006,
 2008: 10.575773063279428,
 2009: 10.366565878065071,
 2010: 10.518374538414921,
 2011: 10.777425399363267,
 2012: 10.852779586683317,
 2013: 10.288876523972055,
 2014: 11.99397934228406,
 2015: 13.324569095540856,
 2016: 13.053336477607926,
 2017: 11.987002285251917,
 2018: 11.08621953828368,
 2019: 12.532831005284974,
 2020: 12.023079755911908,
 2021: 10.629778355236144,
 2022: 10.148767181212008,
 2023: 12.225026793951276,
 2024: 12.770074544269605}

lst_m11_mask = {2001: 14.043164379781057,
 2002: 16.105599388760137,
 2003: 13.483715013004069,
 2004: 14.15739196557732,
 2005: 14.348700439529308,
 2006: 13.85078139530152,
 2007: 14.943424997326689,
 2008: 13.786631099150915,
 2009: 13.648570817407489,
 2010: 14.67904674936639,
 2011: 14.47973201644747,
 2012: 14.111795473988822,
 2013: 13.159061358375626,
 2014: 15.87550866370836,
 2015: 16.388768818139102,
 2016: 16.136909678022384,
 2017: 14.587031772960502,
 2018: 15.22408331042955,
 2019: 15.673831204860631,
 2020: 15.046240662162656,
 2021: 14.308303870126252,
 2022: 12.673677498188717,
 2023: 15.045717196379217,
 2024: 14.955861434161335}

lst_m21_mask = {2001: 14.389877382476167,
 2002: 16.34166828174947,
 2003: 12.570442383767327,
 2004: 14.15592923969709,
 2005: 14.270042157519377,
 2006: 13.029515814794866,
 2007: 15.501608681857977,
 2008: 13.34414955838907,
 2009: 13.46038824900479,
 2010: 13.694979389938803,
 2011: 13.638520347669767,
 2012: 13.090467115172688,
 2013: 13.390383530592992,
 2014: 14.378935532200945,
 2015: 16.153767197798164,
 2016: 14.669666661790536,
 2017: 15.51706892558889,
 2018: 14.485591109650596,
 2019: 15.269387116890934,
 2020: 15.084262305496711,
 2021: 13.885052140705962,
 2022: 13.858707338037501,
 2023: 14.425389763186088,
 2024: 15.741917780725188}

# ERA5

In [8]:
def ERA5_scale_to_celsius(era5_image: ee.Image):
   scaled_lsts = era5_image.select(["mean_2m_air_temperature"]).subtract(273.15).rename(["LST"])
   return era5_image.addBands(srcImg=scaled_lsts, overwrite=True)

era5_coll = ee.ImageCollection("ECMWF/ERA5/MONTHLY").map(lambda img: ERA5_scale_to_celsius(img))

In [9]:
def get_average_lst_era5(era5_coll: ee.ImageCollection, start_time, end_time):
   if(era5_coll.size() == 0):
      return None
   mean_temp = era5_coll.filterDate(start_time, end_time).mean()
   temp = mean_temp.reduceRegion(reducer=ee.Reducer.mean(), geometry=baikal_shape, scale=30, maxPixels=1000000000)
   result_dict = temp.getInfo()
   if(len(result_dict) == 0):
      return None
   return temp.getInfo()['LST']

In [10]:
lst_era5 = {}

for year in range(landsat_start_year, end_year):
   start_date = ee.Date(f"{year}-{openwater_month_start}-1")
   end_date = ee.Date(f"{year}-{openwater_month_end}-1")
   
   result = get_average_lst_era5(era5_coll, start_date, end_date)
   if(result is not None):
      lst_era5[year] = result
      
lst_era5

{1990: 12.483379430280364,
 1991: 12.565505169512653,
 1992: 11.186883593142985,
 1993: 12.541311973364612,
 1994: 12.576303636304328,
 1995: 11.890433619000861,
 1996: 11.525656457796005,
 1997: 12.173449698802962,
 1998: 11.996263889991974,
 1999: 12.075907567343492,
 2000: 13.519539103604885,
 2001: 13.242348478919768,
 2002: 13.661245955571859,
 2003: 12.523946672316857,
 2004: 12.545542116947537,
 2005: 12.843338763379291,
 2006: 11.951816580670009,
 2007: 13.850654928284408,
 2008: 12.804283342875362,
 2009: 12.338139289366348,
 2010: 12.223101488732853,
 2011: 12.737727082097948,
 2012: 12.805936061381141,
 2013: 11.692829028271104,
 2014: 13.24021690561049,
 2015: 14.096213132465529,
 2016: 14.101289846939393,
 2017: 13.00368375620691,
 2018: 13.100451615060136,
 2019: 13.335775986173134,
 2020: 10.744542470002987}

In [ ]:
lst_era5_s = {1990: 12.483379430280364,
 1991: 12.565505169512653,
 1992: 11.186883593142985,
 1993: 12.541311973364612,
 1994: 12.576303636304328,
 1995: 11.890433619000861,
 1996: 11.525656457796005,
 1997: 12.173449698802962,
 1998: 11.996263889991974,
 1999: 12.075907567343492,
 2000: 13.519539103604885,
 2001: 13.242348478919768,
 2002: 13.661245955571859,
 2003: 12.523946672316857,
 2004: 12.545542116947537,
 2005: 12.843338763379291,
 2006: 11.951816580670009,
 2007: 13.850654928284408,
 2008: 12.804283342875362,
 2009: 12.338139289366348,
 2010: 12.223101488732853,
 2011: 12.737727082097948,
 2012: 12.805936061381141,
 2013: 11.692829028271104,
 2014: 13.24021690561049,
 2015: 14.096213132465529,
 2016: 14.101289846939393,
 2017: 13.00368375620691,
 2018: 13.100451615060136,
 2019: 13.335775986173134,
 2020: 10.744542470002987}